# 参考文献

[Visualizing Bert Embeddings](https://krishansubudhi.github.io/deeplearning/2020/08/27/bert-embeddings-visualization.html)

In [ ]:
import torchtext
print(torchtext.__version__)

In [ ]:
print('hello world')

In [ ]:
from google.colab import drive
drive.mount('/content/dirve')

In [ ]:
!pip install transformers fugashi mecab-python3 ipadic

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator

import numpy as np

import random
import math
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchtext
import torch
from torchtext import data
from torchtext import datasets
import pandas as pd

In [ ]:
from transformers import BertJapaneseTokenizer, BertForPreTraining
tok = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

In [ ]:
def tokenizer(text):
  return tok.tokenize(text)

In [ ]:
sent = "今日は雨だけど、明日は晴れそう。今日は全然勝てなかった"

In [ ]:
tokenizer(sent)

In [ ]:
SRC = data.Field(sequential=True, tokenize = tokenizer, init_token='<sos>', eos_token='<eos>', lower = True)
#TRG = data.Field(sequential=True, tokenize = tokenizer, init_token='<sos>', eos_token='<eos>', lower = True)

In [ ]:
"""
train, val, test = data.TabularDataset.splits(
        path="/content/dirve/My Drive/Colab Notebooks/data/", train='train.tsv',
        validation='val.tsv', test='test.tsv', format='tsv',
        fields=[('SRC', SRC), ('TRG', SRC)])
"""

In [ ]:
train, val, test = data.TabularDataset.splits(
        path="/content/dirve/My Drive/Colab Notebooks/data/", train='one_train.tsv',
        validation='one_val.tsv', test='one_test.tsv', format='tsv',
        fields=[('SRC', SRC), ('TRG', SRC)])

In [ ]:
SRC.build_vocab(train)
#TRG.build_vocab(train)

In [ ]:
bert_model = BertForPreTraining.from_pretrained(
    "cl-tohoku/bert-base-japanese", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = True, # 隠れ層を出力するか
)

In [ ]:
bert_model.resize_token_embeddings(len(SRC.vocab))

In [ ]:
bert_model.get_input_embeddings()

In [ ]:
bert_model.get_output_embeddings()

In [ ]:
"""
train_batch_size = 50
test_batch_size = 10
eval_batch_size = 2
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), sort = False,  batch_sizes = (train_batch_size,eval_batch_size, test_batch_size), device= device)]
"""

In [ ]:
train_batch_size = 100
test_batch_size = 2
eval_batch_size = 100
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), sort = False,  batch_sizes = (train_batch_size,eval_batch_size, test_batch_size), device= device)

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()
    self.hid_dim = hid_dim
    self.n_layers = n_layers
    #self.embedding = nn.Embedding(input_dim, emb_dim)
    self.embedding = bert_model.get_input_embeddings()
    self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src):
    embedded = self.dropout(self.embedding(src))
    outputs, (hidden, cell) = self.rnn(embedded)
    return hidden, cell

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()
    self.output_dim = output_dim
    self.hid_dim = hid_dim
    self.n_layers = n_layers

    #self.embedding = nn.Embedding(output_dim, emb_dim)
    self.embedding = bert_model.get_input_embeddings()
    self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

    #self.fc_out = nn.Linear(hid_dim, output_dim)
    self.fc_out = bert_model.get_output_embeddings()
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, input, hidden, cell):
    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))
    output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
    #print(output.squeeze(0).size())
    prediction = self.fc_out(output.squeeze(0))

    return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
    batch_size = trg.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    hidden, cell = self.encoder(src)

    output = trg[0,:]
    #print(input)

    for t in range(1, trg_len):
      output, hidden, cell = self.decoder(output, hidden, cell)

      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      output = (trg[t] if teacher_force else top1)
    
    return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
# OUTPUT_DIM = 3996
OUTPUT_DIM = 3454
ENC_EMB_DIM = 768
DEC_EMB_DIM = 768
ENC_HID_DIM = 768
DEC_HID_DIM = 768
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()

  epoch_loss = 0

  for i, batch in enumerate(iterator):
    src = batch.SRC
    trg = batch.TRG
    optimizer.zero_grad()

    output = model(src, trg)

    output_dim = output.shape[-1]
    output = output[:].view(-1, output_dim)
    trg = trg[:].view(-1)

    loss = criterion(output, trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm(model.parameters(), clip)
    optimizer.step()

    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
  model.eval()

  epoch_loss = 0

  with torch.no_grad():

    for i, batch in enumerate(iterator):

      src = batch.SRC
      trg = batch.TRG

      output = model(src, trg)

      output_dim = output.shape[-1]

      output = output[:].view(-1, output_dim)
      trg = trg[:].view(-1)

      loss = criterion(output, trg)
      epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins*60))
  return elapsed_mins, elapsed_secs

In [ ]:
epochs = 100
clip = 1
#def  model_train(epochs, clip):
best_valid_loss = float('inf')
best_model = None

for epoch in range(epochs):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, clip)
    valid_loss = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        #torch.save(model.state_dict(), '/content/dirve/My Drive/Colab Notebooks/model/bert_embedded_seq2seq.pth')
        best_model = model
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [ ]:
torch.save(best_model.state_dict(), '/content/dirve/My Drive/Colab Notebooks/model/bert_embedded_seq2seq.pth')

In [ ]:
model.state_dict(torch.load("/content/dirve/My Drive/Colab Notebooks/model/bert_embedded_seq2seq.pth"))

In [ ]:
def gen_sentence(sentence, src_field, trg_field, model, batch_size):
  model.eval()
  in_str, out_str, pred, tmp = [], [], [], []
  length = len(sentence)

  with torch.no_grad():
    for _, batch in enumerate(sentence):
      src = batch.SRC
      trg = batch.TRG
      output = model(src, trg)
          
      for j in range(min(length, batch_size)):
        _, topi = output.data.topk(1)
        _, topi_s = output.data.topk(2) 
        for k in range(topi.size()[1]):
          if topi[:, k][0] == trg_field.vocab.stoi["<eos>"]:
            for m in range(topi_s.size()[0]):
              for l in range(topi_s.size()[1]):
                topi[m][l][0] = topi_s[m][l][1]
          for i in range(topi.size()[0]):
            if trg_field.vocab.itos[topi[:, k][i]] == "<eos>":
              break
            tmp.append(trg_field.vocab.itos[topi[:, k][i]])
          pred.append(tmp)
          tmp = []
        #print(src.size())
        in_str.append([src_field.vocab.itos[i.item()] for i in src[:,j] if src_field.vocab.itos[i.item()] != "<eos>"])
        out_str.append([trg_field.vocab.itos[i.item()] for i in trg[:,j] if trg_field.vocab.itos[i.item()] != "<eos>"])
      
  return in_str, out_str, pred

In [ ]:
# 中間発表時にはテストデータは用いない
test_in, test_out, test_pred = [],[],[]
test_in, test_out, test_pred = gen_sentence(test_iter, SRC, SRC, best_model, test_batch_size)

In [ ]:
val_in, val_out, val_pred = [],[],[]
val_in, val_out, val_pred = gen_sentence(val_iter, SRC, SRC, model, eval_batch_size)

In [ ]:
train_in, train_out, train_pred = [],[],[]
train_in, train_out, train_pred = gen_sentence(train_iter, SRC, SRC, model, train_batch_size)

In [ ]:
import pandas as pd
def convert_list_to_df(in_list, out_list, pred_list):
  row = []
  pred = ""
  for i in range(len(in_list)):
    batch_input = in_list[i]
    batch_output = out_list[i]
    batch_pred = pred_list[i]
    input = [j for j in batch_input if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>" and j != "[unk]"]
    output = [j for j in batch_output if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>" and j != "[unk]"]
    predict = [j for j in batch_pred if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>" and j != "[unk]"]
    input_str = "".join(input).replace("#", "")
    output_str ="".join(output).replace("#", "")
    predict_str = "".join(predict).replace("#", "")
    row.append([input_str, output_str, predict_str])
    pred = ""
  df = pd.DataFrame(row, columns=["input","answer","predict"])
  df = df.sort_values('input')
  return df

In [ ]:
train_df = convert_list_to_df(train_in, train_out, train_pred)
val_df = convert_list_to_df(val_in, val_out, val_pred)
test_df = convert_list_to_df(test_in, test_out, test_pred)

In [ ]:
df_s = pd.concat([train_df, test_df])
df_s = df_s.sort_values("input")

In [ ]:
# df_s.to_csv("/content/dirve/My Drive/Colab Notebooks/csv/result_bertSeq2seq.csv")

In [ ]:
df_s.to_csv("/content/dirve/My Drive/Colab Notebooks/csv/one_result_bertSeq2seq.csv")